In [ ]:
from delta.tables import *
# import datetime 
import time

# dt = str(datetime.date.today())
year, month, day = time.strftime("%Y"), time.strftime("%m"), time.strftime("%d")
hour, minute, second = time.strftime("%H"), time.strftime("%M"), time.strftime("%S")

columns = ["db_name", "schema_name", "table_name", "incremental_column", "incremental_column_value", "merge_key_column", "delta_source_path"]
values = [("lakehouse", "dbo", "products", "updated_at", "1900-01-01 00:00:00.000", "product_code",""),
       ("lakehouse", "dbo", "store_customers", "updated_at", "1900-01-01 00:00:00.000", "customer_id",""),
       ("lakehouse", "dbo", "store_orders", "updated_at", "1900-01-01 00:00:00.000", "orders_number",""),
	   ("lakehouse", "dbo", "inventory", "updated_at", "1900-01-01 00:00:00.000", "product","")]

if not spark.catalog.tableExists("bronze_control_table"):
    DeltaTable.createIfNotExists(spark) \
        .tableName("bronze_control_table") \
        .addColumn("db_name", "STRING") \
        .addColumn("schema_name", "STRING") \
        .addColumn("table_name", "STRING") \
        .addColumn("incremental_column", "STRING") \
        .addColumn("incremental_column_value", "TIMESTAMP", comment = "updated on each run") \
        .addColumn("merge_key_column", "STRING") \
        .addColumn("delta_source_path", "STRING", comment = "updated on each run") \
        .execute()
        
    df = spark.createDataFrame(values, columns)
    df = df.withColumn("incremental_column_value",df.incremental_column_value.cast("timestamp"))
    df.write.mode("append").format("delta").saveAsTable("bronze_control_table")

In [ ]:
def get_tables_list():
    _df = spark.read.table("bronze_control_table")
    display(_df)
    tbls = _df.select("table_name").collect()

    return tbls


In [ ]:
def create_and_update_delta_source_path(_table, _year, _month, _day, _hour, _minute):
    path = f"abfss://Training@onelake.dfs.fabric.microsoft.com/trg_lakehouse.Lakehouse/Files/bronze/{_table}/processed/{_year}/{_month}/{_day}/{_hour}/{_minute}"
   
    try:
        if not mssparkutils.fs.exists(path):
            print(f"path does not exit, creating path: {path}")
            mssparkutils.fs.mkdirs(path)
        else:
            print(f"path: {path} already exists")
    except Exception as e:
        print(e)

    # folder = f"abfss://Training@onelake.dfs.fabric.microsoft.com/trg_lakehouse.Lakehouse/Files/bronze/{_table}/processed/{_year}/{_month}/{_day}/{_hour}/{_minute}"
    folder = f"bronze/{_table}/processed/{_year}/{_month}/{_day}/{_hour}/{_minute}"
   
    query = f"UPDATE bronze_control_table SET delta_source_path = '{folder}' WHERE table_name = '{_table}'"
    spark.sql(query)


In [ ]:
tables_to_process = get_tables_list()
for row in tables_to_process:
    table = row["table_name"]
    create_and_update_delta_source_path(table, year, month, day, hour, minute)


In [ ]:
%%sql

SELECT table_name, delta_source_path FROM bronze_control_table
